In [10]:
import pygmsh
import meshio
import pygalmesh
import hoomd
import hoomd.md
import numpy as np
import copy
import optimesh
from mshr import *
from dolfin import *
from collections import Counter
import matplotlib.pyplot as plt
import os

# Run Parameters

In [ ]:
lengths=[np.linalg.norm(InputMesh.points[bond[0]]- InputMesh.points[bond[1]]) for bond in bonds]

In [ ]:
plt.hist(lengths)

# Make a basic mesh

## Dolfin

In [4]:
def MakeDolfinMesh(a):
    # make the mesh. Lets have a unit circle. It seems, from trial and error, that
    # res = 1.5*Radius/mesh_size,
    domain = Ellipse(Point(0, 0, 0),1.0,1.0, 40)
    mesh = generate_mesh(domain, 1.5/a)
    mesh.init()
    
    # need to add a 3rd dimension 0 coordinate here
    points = np.insert(mesh.coordinates(),2,0,axis=1)
    cells = cells = [("triangle",mesh.cells() )]
    InputMesh = meshio.Mesh(points,cells)
    #copy for modifying at output
    OutputMesh=copy.deepcopy(InputMesh)
    
    # make list of:
    #interior bonds : interiorbonds
    # edge bonds :edgepoints
    # bonds : interiorbonds+edgebonds
    # angle triples: angletriples
    mesh.cells()
    triangles=mesh.cells()
    x = [[[triangle[0],triangle[1]],[triangle[0],triangle[2]],[triangle[1],triangle[2]] ]   for triangle in triangles]
    flattenedx = [val for sublist in x for val in sublist]
    bonds = [[x[0],x[1]] if x[0]<x[1] else [x[1],x[0]] for x in flattenedx]

    # get a list of the bonds on the edge, and in the interior
    edgebonds=[]
    interiorbonds=[]
    for elem in bonds:
        if 1==bonds.count(elem):
            edgebonds.append(elem)
        elif 2==bonds.count(elem) and elem not in interiorbonds:
            interiorbonds.append(elem)

    bonds=interiorbonds+edgebonds

    # for the edge bonds, get the angle triples
    EdgeVertices = list(set([val for sublist in edgebonds for val in sublist]))
    angletriples=[]

    for vertex in EdgeVertices:
        Neighbors=[x for x in edgebonds if vertex in x]
        NeighborVertices = [val for sublist in Neighbors for val in sublist if val!=vertex]
        angletriples.append([NeighborVertices[0],vertex,NeighborVertices[1]])
        
    return InputMesh, OutputMesh, interiorbonds,edgebonds,angletriples


## pygmesh

In [ ]:
def MakePygMesh():
    
    with pygmsh.occ.Geometry() as geom:
        geom.characteristic_length_max = 1
        geom.add_disk([0.0, 0.0], radius0=10.0,radius1=9.0,mesh_size=0.5),
        InputMesh = geom.generate_mesh()
    
    OutputMesh=copy.deepcopy(InputMesh)
    
    # make list of:
    #interior bonds : interiorbonds
    #edge bonds :edgepoints
    # angle triples: angletriples

    triangles=InputMesh.cells[1].data
    x = [[[triangle[0],triangle[1]],[triangle[0],triangle[2]],[triangle[1],triangle[2]] ]   for triangle in triangles]
    flattenedx = [val for sublist in x for val in sublist]
    orderedx = [[x[0],x[1]] if x[0]<x[1] else [x[1],x[0]] for x in flattenedx]
    bonds = []

    bonds = []
    for elem in orderedx:
        if elem not in bonds:
            bonds.append(elem)

    # sort the edge bonds
    edgebonds = [[x[0],x[1]] if x[0]<x[1] else [x[1],x[0]] for x in InputMesh.cells[0].data]
    interiorbonds=[x for x in bonds if x not in edgebonds]
    bonds=interiorbonds+edgebonds

    # Angle triples
    NEdgeBonds= len(edgebonds)
    angletriples=[[(i-1)%NEdgeBonds,i,(i+1)%NEdgeBonds] for i in range(0,NEdgeBonds) ]
    
    return InputMesh, OutputMesh, interiorbonds,edgebonds,angletriples
    

 # Initialising the HOOMD simulation 

User settings: What are the continuum parameters we want? where shall we store the run data?

In [24]:
# Target mesh size:
target_a = 0.05
# continuum bending modulus:
kc=1
# continuum shear modulus:
mu=1
# root folder for data
DataFolder='/home/jackbinysh/Code/ActiveElastocapillarity/Data/'
# Folder for the run data
RunFolder="Test/"
# Name of the run
RunName=""

In [25]:
path = DataFolder+RunFolder
try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)

Successfully created the directory /home/jackbinysh/Code/ActiveElastocapillarity/Data/Test/ 


Make the mesh

In [28]:
InputMesh, OutputMesh, interiorbonds,edgebonds,angletriples = MakeDolfinMesh(target_a)bb
InputMesh.write(DataFolder+RunFolder+RunName+"InputMesh.vtk")
#InputMesh, OutputMesh, interiorbonds,edgebonds,angletriples = MakePygMesh()

 ## HOOMD stuff

In [ ]:
hoomd.context.initialize("");

Define the snapshot. We will have a unique bond id for every bond in the system, as they will all have different rest lengths. We also want to make a distinction between surface and bulk bonds.

In [ ]:
# number of points and bonds
Npts=len(InputMesh.points);
bonds = interiorbonds+edgebonds
NBonds = len(bonds);
#indices
bondindices = list(range(0,NBonds))
#surface bond or not
bondclassification = [0]*len(interiorbonds)+[1]*len(edgebonds)

In [ ]:
snapshot = hoomd.data.make_snapshot(N=Npts
                                    ,box=hoomd.data.boxdim(Lx=200, Ly=200,dimensions=2)
                                    ,particle_types=['A']
                                    ,bond_types=[str(i) for i in  bondindices]
                                    ,angle_types=['0']
                                   );

Read in the points, bonds and angles

In [ ]:
# points
snapshot.particles.position[:] = InputMesh.points;
snapshot.particles.typeid[0:Npts]=0
# bonds
snapshot.bonds.resize(NBonds)
snapshot.bonds.group[:] = bonds
snapshot.bonds.typeid[:] = bondindices
#angle triples
snapshot.angles.resize(len(angletriples))
snapshot.angles.group[:] = angletriples
snapshot.angles.typeid[:len(angletriples)] =0

In [ ]:
system=hoomd.init.read_snapshot(snapshot);

Right, lets define the bond type and parameters for each bond. In 2D, we know that the elastic modulii are proportional to the microscopic spring constant. We also know that the continuum and microscopic momdulii are related by a lattice space: $\mu = O(1) k$, $k_c = k_d a$

In [ ]:
kd=kc/target_a
k = mu

In [ ]:
harmonic = hoomd.md.bond.harmonic();

for i in snapshot.bonds.typeid:
    p1,p2 = snapshot.bonds.group[i]
    
    if(0==bondclassification[i]):  
        restlength=np.linalg.norm(InputMesh.points[p2] - InputMesh.points[p1])
        harmonic.bond_coeff.set(str(i), k=k, r0=restlength);

    if(1==bondclassification[i]): 
        restlength=np.linalg.norm(InputMesh.points[p2] - InputMesh.points[p1])
        harmonic.bond_coeff.set(str(i), k=1, r0=40*restlength);

angle = hoomd.md.angle.harmonic();
angle.angle_coeff.set('0', k=kd, t0=np.pi-0*((2*np.pi)/len(edgebonds)) );

Define the integrator. In this case, a langevin dynamics

In [ ]:
hoomd.md.integrate.mode_standard(dt=0.001);
all = hoomd.group.all();
integrator = hoomd.md.integrate.langevin(group=all,kT=0,seed=0,dscale=2);

Define a callback, which we want run periodically

In [ ]:
class WritePositions:
    def __init__(self, system):
        self.system = system;
    def __call__(self, timestep):
        snap = self.system.take_snapshot();
        OutputMesh.points = snap.particles.position
        OutputMesh.write("Data/"+RunString+str(timestep)+".vtk")
        
hoomd.analyze.callback(callback=WritePositions(system), period=5000);

# Run the simulation

In [ ]:
hoomd.run(1000000);